In [ ]:
import pandas as pd
import numpy as np
import ast
import json

# reads string and evaluates str(string), if true then it will return back the string
def parse_tuple(string):
    try:
        s = ast.literal_eval(str(string))
        if type(s) == tuple:
            return s
        return
    except:
        return


# reads json files, for features, from outputs of other notebooks
# name update dictionaries 
with open('investor_country_name_grouping_v2.json') as json_file:
    investor_country_name_dict = json.load(json_file)

with open('startup_country_name_grouping_v2.json') as json_file:
    startup_country_name_dict = json.load(json_file)

with open('investor_name_grouping_v2.json') as json_file:
    investor_name_dict = json.load(json_file)


# index dictionaries
with open('rounds_index_v2.json') as json_file:
    rounds_index = json.load(json_file)

with open('investor_country_round_index_v2.json') as json_file:
    investor_country_round_index = json.load(json_file)

with open('startup_country_round_index_v2.json') as json_file:
    startup_country_round_index = json.load(json_file)

with open('investors_rounds_v2.json') as json_file:
    investors_rounds_index = json.load(json_file)

In [ ]:
data = pd.read_csv('rounds_training_v2.csv') # dataframe from the training dataset of the rounds csv
del data['Unnamed: 0']
data.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B
1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B
2,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A
3,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture
4,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed


In [ ]:
data = data[data['raised_amount_usd'].notnull()].copy() # removing nulls
data = data[data['raised_amount_usd'] >= 50000].copy() # arbitrary threshold for the funding
data.shape

(69616, 13)

In [ ]:
investment = pd.read_csv('investments_training_v2.csv') # investment dataset dataframe
del investment['Unnamed: 0']
investment.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,CHN,23,Shanghai,Shanghai,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A
1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,DEU,5,Frankfurt,Frankfurt,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture
2,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,CHL,12,Santiago,Las Condes,/organization/start-up-chile,Start-Up Chile,CHL,12,Santiago,Santiago,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed
3,/organization/1-800-dentist,1-800-DENTIST,Health and Wellness,USA,CA,Los Angeles,Los Angeles,/organization/bain-capital-ventures,Bain Capital Ventures,USA,MA,Boston,Boston,/funding-round/5274aacc211163fc7c86539ce94bbacc,undisclosed,NaN,2010-08-19,NaN,undisclosed
4,/organization/1-800-dentist,1-800-DENTIST,Health and Wellness,USA,CA,Los Angeles,Los Angeles,/organization/cue-ball,Cue Ball,USA,MA,Boston,Boston,/funding-round/5274aacc211163fc7c86539ce94bbacc,undisclosed,NaN,2010-08-19,NaN,undisclosed


In [ ]:
# merge the investment dataframe with rounds dataframe
data_merged = data.merge(investment[['investor_name', 'investor_country_code', 'funding_round_permalink']]
                ,how = 'inner', on = 'funding_round_permalink')
data_merged.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,DT Capital Partners,CHN
1,/organization/1-mainstream,1 Mainstream,Apps|Cable|Distribution|Software,USA,CA,SF Bay Area,Cupertino,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,2015-03-17,5000000.0,venture,Sky,NaN
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,FF Angel LLC,USA
3,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,True Ventures,USA
4,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,Jon Callaghan,USA


In [ ]:
data_merged = data_merged[data_merged['investor_country_code'].notnull()] # removes nulls
data_merged.shape

(92231, 15)

#### Applying Investor Country Name grouping

In [ ]:
# merge investor_country_code with the dictionary from the json files, which will replace countries to "OTHER"
data_merged.loc[:,'investor_country_code'] = data_merged['investor_country_code'].apply(lambda x: investor_country_name_dict['investor_country_name_new'][x])
data_merged.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,DT Capital Partners,CHN
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,FF Angel LLC,USA
3,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,True Ventures,USA
4,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,Jon Callaghan,USA
5,/organization/1000chi,Beijing 1000CHI Software Technology,Mobile,NaN,NaN,NaN,NaN,/funding-round/854fae13c5b26b2ea6b32b06c2e5890c,angel,NaN,2010-04-01,43923865.0,angel,Sina Weibo,CHN


#### Applying Investor Name grouping

In [ ]:
# update some investor names as 'Other'
data_merged.loc[:,'investor_name'] = data_merged['investor_name'].apply(lambda x: investor_name_dict[x])
data_merged.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,DT Capital Partners,CHN
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,FF Angel LLC,USA
3,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,True Ventures,USA
4,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,Jon Callaghan,USA
5,/organization/1000chi,Beijing 1000CHI Software Technology,Mobile,NaN,NaN,NaN,NaN,/funding-round/854fae13c5b26b2ea6b32b06c2e5890c,angel,NaN,2010-04-01,43923865.0,angel,Other,CHN


#### Creating Investor Country & Round Index feature

In [ ]:
# creates the index data frame from the dictionary from the json file
investor_country_round_index = pd.DataFrame.from_dict(investor_country_round_index, orient='index')

# calls the previously defined "parse_tuple" function to parse through 
investor_country_round_index.index = [parse_tuple(item) for item in list(investor_country_round_index.index)]


investor_country_round_index.index = pd.MultiIndex.from_tuples(investor_country_round_index.index)
investor_country_round_index.index.names = ['country', 'round']
print(investor_country_round_index)


                               0
country round                   
ARG     angel                  1
        convertible_note       1
        product_crowdfunding   3
        seed                   1
        undisclosed            3
...                           ..
USA     venture D              8
        venture E              9
        venture F              9
        venture G             10
        venture H             10

[521 rows x 1 columns]


In [ ]:
#since searching in dictionary objects are way faster, index df is converted to dict
investor_country_round_index_dict = investor_country_round_index.to_dict()[0]
investor_country_round_index_dict

{('ARG', 'angel'): '1',
 ('ARG', 'convertible_note'): '1',
 ('ARG', 'product_crowdfunding'): '3',
 ('ARG', 'seed'): '1',
 ('ARG', 'undisclosed'): '3',
 ('ARG', 'venture'): '4',
 ('ARG', 'venture A'): '4',
 ('ARG', 'venture C'): '7',
 ('AUS', 'angel'): '2',
 ('AUS', 'convertible_note'): '1',
 ('AUS', 'debt_financing'): '9',
 ('AUS', 'non_equity_assistance'): '1',
 ('AUS', 'post_ipo_equity'): '2',
 ('AUS', 'private_equity'): '10',
 ('AUS', 'seed'): '2',
 ('AUS', 'undisclosed'): '4',
 ('AUS', 'venture'): '5',
 ('AUS', 'venture A'): '4',
 ('AUS', 'venture B'): '6',
 ('AUS', 'venture C'): '6',
 ('AUS', 'venture D'): '7',
 ('AUS', 'venture E'): '8',
 ('AUS', 'venture F'): '9',
 ('BEL', 'angel'): '2',
 ('BEL', 'grant'): '1',
 ('BEL', 'private_equity'): '8',
 ('BEL', 'seed'): '2',
 ('BEL', 'undisclosed'): '4',
 ('BEL', 'venture'): '5',
 ('BEL', 'venture A'): '5',
 ('BEL', 'venture B'): '6',
 ('BEL', 'venture C'): '7',
 ('BEL', 'venture D'): '8',
 ('BEL', 'venture E'): '10',
 ('BEL', 'venture F

In [ ]:
def inv_country_rnd_idx(row):
    country = row['investor_country_code']
    round_ = row['funding_round']

    search_key = tuple((country, round_))
    
    value = investor_country_round_index_dict[search_key]
    #a = investor_country_round_index[
    #    (investor_country_round_index.index.get_level_values('country') == country) &
    #    (investor_country_round_index.index.get_level_values('round') == round_)][0]

    return int(value)
    #return int(a.values[0])
    

data_merged['investor_country_round_index'] = data_merged.apply(inv_country_rnd_idx, axis = 1)
data_merged.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code,investor_country_round_index
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,DT Capital Partners,CHN,6
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,FF Angel LLC,USA,3
3,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,True Ventures,USA,3
4,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,Jon Callaghan,USA,3
5,/organization/1000chi,Beijing 1000CHI Software Technology,Mobile,NaN,NaN,NaN,NaN,/funding-round/854fae13c5b26b2ea6b32b06c2e5890c,angel,NaN,2010-04-01,43923865.0,angel,Other,CHN,4


#### Creating Investor & Round Index feature

In [ ]:
investors_rounds_index = pd.DataFrame.from_dict(investors_rounds_index, orient='index')
investors_rounds_index.index = [parse_tuple(item) for item in list(investors_rounds_index.index)]
investors_rounds_index.index = pd.MultiIndex.from_tuples(investors_rounds_index.index)
investors_rounds_index.index.names = ['investor', 'round']
investors_rounds_index

0
investor      round              
.406 Ventures seed              4
              venture           6
              venture A         5
              venture B         7
              venture C         7
...                            ..
yet2Ventures  venture E         8
zPark Capital angel             1
              convertible_note  1
              seed              2
              venture B         8

[20824 rows x 1 columns]

In [ ]:
investors_rounds_index_dict = investors_rounds_index.to_dict()[0]
investors_rounds_index_dict

{('.406 Ventures', 'seed'): '4',
 ('.406 Ventures', 'venture'): '6',
 ('.406 Ventures', 'venture A'): '5',
 ('.406 Ventures', 'venture B'): '7',
 ('.406 Ventures', 'venture C'): '7',
 ('.406 Ventures', 'venture D'): '8',
 ('.406 Ventures', 'venture E'): '9',
 ('.406 Ventures', 'venture F'): '10',
 ('101 Startups', 'angel'): '1',
 ('101 Startups', 'seed'): '1',
 ('101 Startups', 'venture A'): '1',
 ('10X Fund LP', 'venture B'): '1',
 ('10Xelerator', 'grant'): '1',
 ('10Xelerator', 'seed'): '1',
 ('10Xelerator', 'undisclosed'): '1',
 ('123 Venture', 'venture'): '3',
 ('123 Venture', 'venture B'): '6',
 ('123Venture', 'venture'): '6',
 ('123Venture', 'venture A'): '3',
 ('123Venture', 'venture B'): '5',
 ('123Venture', 'venture C'): '5',
 ('123Venture', 'venture D'): '8',
 ('137 Ventures', 'secondary_market'): None,
 ('137 Ventures', 'venture C'): '10',
 ('14W', 'angel'): '1',
 ('14W', 'seed'): '3',
 ('14W', 'venture'): '6',
 ('14W', 'venture A'): '5',
 ('14W', 'venture B'): '8',
 ('14W',

In [ ]:
def inv_rnd_idx(row):
    investor = row['investor_name']
    round_ = row['funding_round']
    
    search_key = tuple((investor, round_))

    value = investors_rounds_index_dict[search_key]
    return int(value)


    #if round_ == 'venture':
    #    round_concat = round_ + ' ' + str(round_name)
    #    a = investors_rounds_index[
    #        (investors_rounds_index.index.get_level_values('investor') == investor) &
    #        (investors_rounds_index.index.get_level_values('round') == round_concat)][0]
    #    return int(a.values[0])
    #else:
    #    a = investors_rounds_index[
    #        (investors_rounds_index.index.get_level_values('investor') == investor) &
    #        (investors_rounds_index.index.get_level_values('round') == round_)][0]
    #    return int(a.values[0])

    
    

data_merged['investors_round_index'] = data_merged.apply(inv_rnd_idx, axis = 1)
data_merged.head(50)

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code,investor_country_round_index,investors_round_index
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,DT Capital Partners,CHN,6,6
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,FF Angel LLC,USA,3,2
3,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,True Ventures,USA,3,3
4,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,Jon Callaghan,USA,3,1
5,/organization/1000chi,Beijing 1000CHI Software Technology,Mobile,NaN,NaN,NaN,NaN,/funding-round/854fae13c5b26b2ea6b32b06c2e5890c,angel,NaN,2010-04-01,43923865.0,angel,Other,CHN,4,1
6,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/01c1aa44619ae45a16e3098fc73facfa,debt_financing,NaN,2013-04-08,647879.0,debt_financing,Alliance of Angels,USA,7,1
8,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/01c1aa44619ae45a16e3098fc73facfa,debt_financing,NaN,2013-04-08,647879.0,debt_financing,Scout Ventures,USA,7,1
12,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,NaN,2014-01-15,2906832.0,venture,Converge Venture Partners,USA,6,4
13,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/89317984885b44f02e4befb9dc2e588c,venture,NaN,2014-08-01,1500000.0,venture,Converge Venture Partners,USA,6,4
14,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/fd1b3064232ba9b3cfb5653a619e82e8,venture,NaN,2011-04-05,642000.0,venture,Alliance of Angels,USA,6,4


#### Updating indexes with non unique values for each startup and round pair
In one investment round, there might be more than one investor companies. This means for each startup company and round pair, investor country round index and investor round index might not be unique. To represent these indexes, an aggragation is applied as grouping each startup company and round pair, updating these index features with mean index value.

In [ ]:
data_merged_grouped = data_merged.groupby(['funding_round_permalink'])\
                    .agg({'investor_country_round_index': 'mean',
                          'investors_round_index': 'mean'}).round()
del data_merged['investor_country_round_index'], data_merged['investors_round_index']
data_merged = data_merged.merge(data_merged_grouped, how = 'left', left_on = 'funding_round_permalink', right_index=True)
data_merged['investor_country_round_index'] = data_merged['investor_country_round_index'].astype(int)
data_merged['investors_round_index'] = data_merged['investors_round_index'].astype(int)
data_merged.tail(50)

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_name,investor_country_code,investor_country_round_index,investors_round_index
113180,/organization/zyncro,Zyncro,Cloud Computing|Enterprise 2.0|Enterprise Soft...,ESP,56,Barcelona,Barcelona,/funding-round/35310bcb0945d9d4b5ffa5482e5c87f3,venture,B,2012-02-03,2105600.0,venture B,Caixa Capital Risc,ESP,6,4
113181,/organization/zyncro,Zyncro,Cloud Computing|Enterprise 2.0|Enterprise Soft...,ESP,56,Barcelona,Barcelona,/funding-round/c8dd9ae33cc5d5a1d972b470a5955373,venture,A,2011-04-05,1699920.0,venture A,Cabiedes & Partners,ESP,4,4
113182,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/1bc4f7b3a778e5abfbd7131f595f5011,venture,B,2009-11-04,15187000.0,venture B,Andreessen Horowitz,USA,7,9
113183,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/1bc4f7b3a778e5abfbd7131f595f5011,venture,B,2009-11-04,15187000.0,venture B,DST Global,RUS,7,9
113184,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/1bc4f7b3a778e5abfbd7131f595f5011,venture,B,2009-11-04,15187000.0,venture B,IVP,USA,7,9
113185,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/1bc4f7b3a778e5abfbd7131f595f5011,venture,B,2009-11-04,15187000.0,venture B,Tiger Global Management,USA,7,9
113186,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/1bc4f7b3a778e5abfbd7131f595f5011,venture,B,2009-11-04,15187000.0,venture B,Kevin Rose,USA,7,9
113187,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/66d64809546fe2031d3479161e3c88f9,venture,A,2008-01-01,10000000.0,venture A,Avalon Ventures,USA,5,5
113188,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/66d64809546fe2031d3479161e3c88f9,venture,A,2008-01-01,10000000.0,venture A,Other,USA,5,5
113189,/organization/zynga,Zynga,Facebook Applications|Games|Networking|Technology,USA,CA,SF Bay Area,San Francisco,/funding-round/66d64809546fe2031d3479161e3c88f9,venture,A,2008-01-01,10000000.0,venture A,Foundry Group,USA,5,5


In [ ]:
# displays the distribution
data_merged['investor_country_round_index'].value_counts().sort_index().plot.bar()

<AxesSubplot:>

In [ ]:
# displays the distribution
data_merged['investors_round_index'].value_counts().sort_index().plot.bar()

<AxesSubplot:>

In [ ]:
data_merged.shape

(92231, 17)

In [ ]:
# drops these columns because they are now indices
del data_merged['investor_name'], data_merged['investor_country_code']

In [ ]:
# drop duplicates
data_merged.drop_duplicates(inplace=True)
data_merged.shape

(38157, 15)

In [ ]:
data_merged.head(50)

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,investor_country_round_index,investors_round_index
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,6,6
2,/organization/1000-markets,1000 Markets,Art|E-Commerce|Marketplaces,USA,WA,Seattle,Seattle,/funding-round/ce6749b6441f52ccaf38b6d5578779fd,seed,NaN,2009-05-15,500000.0,seed,3,2
5,/organization/1000chi,Beijing 1000CHI Software Technology,Mobile,NaN,NaN,NaN,NaN,/funding-round/854fae13c5b26b2ea6b32b06c2e5890c,angel,NaN,2010-04-01,43923865.0,angel,4,1
6,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/01c1aa44619ae45a16e3098fc73facfa,debt_financing,NaN,2013-04-08,647879.0,debt_financing,7,1
12,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/6aeb32ab3fc05db2b606d7a2466e8c09,venture,NaN,2014-01-15,2906832.0,venture,6,4
13,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/89317984885b44f02e4befb9dc2e588c,venture,NaN,2014-08-01,1500000.0,venture,6,4
14,/organization/1000museums-com,1000museums.com,Curated Web,USA,MA,MA - Other,Lenox,/funding-round/fd1b3064232ba9b3cfb5653a619e82e8,venture,NaN,2011-04-05,642000.0,venture,6,4
17,/organization/1001-menus,ZenChef,Local Businesses|Restaurants,FRA,A8,Paris,Paris,/funding-round/6364929e6f6c94921f9c853f53b10b7a,seed,NaN,2012-12-15,390000.0,seed,3,2
23,/organization/1001-menus,ZenChef,Local Businesses|Restaurants,FRA,A8,Paris,Paris,/funding-round/645b4cd895450b8e0268027cd7813047,venture,B,2015-11-04,6602694.0,venture B,6,6
26,/organization/1001-menus,ZenChef,Local Businesses|Restaurants,FRA,A8,Paris,Paris,/funding-round/8025123b90c1ab687f9857ffe18bfd3b,venture,A,2013-11-13,1736910.0,venture A,5,4


In [ ]:
data_merged.shape

(38157, 15)

## Creating Sector Round Index

In [ ]:
def drop_dupes(x):
    """this function removes duplicates"""
    return list(dict.fromkeys(x))

# reads the csv from the manually edited categories to sectors
categories = pd.read_csv("company_categories.csv", header=None)
# creates the dictionary from the csv
sectors = categories.set_index(0).to_dict()[1]

In [ ]:
rounds = pd.read_csv("rounds_training_v2.csv")

In [ ]:
rounds.head()

,Unnamed: 0,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round
0,0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B
2,2,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A
3,3,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture
4,4,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed


In [ ]:
# drops NaNs from the category list
rounds = rounds[rounds['company_category_list'].notna()]
# resets the index
rounds = rounds.reset_index(drop=True)
# new category list so it can be split
category_list = rounds['company_category_list']

In [ ]:
categories_list = category_list.tolist()
categories = []
for i in range(len(categories_list)):
    if "|" in str(categories_list[i]):
        categories.append(categories_list[i].split('|')) # splits the categories in that row 
    else:
        categories.append([categories_list[i]]) # makes sure that all the types are consistent, i.e. a list. 

In [ ]:
# drops the original category list
rounds.drop(columns='company_category_list',inplace=True)
# makes a new category list from the split one
rounds['company_category_list'] = categories

In [ ]:
# changes categories into sectors
company_sector = rounds['company_category_list'].apply(lambda x: [sectors[x[i]] for i in range(len(x))])

In [ ]:
# removes duplicates from the sectors
no_dupes_company_sector = company_sector.apply(lambda x: drop_dupes(x))

In [ ]:
# applies back the company sector list to the main dataframe
rounds['company_sector'] = no_dupes_company_sector

In [ ]:
sectors_list = no_dupes_company_sector

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binarized_sectors = mlb.fit_transform(sectors_list)
binarized_sectors_column_names = mlb.classes_

overall_categories = pd.DataFrame(data=binarized_sectors,columns=binarized_sectors_column_names)

In [ ]:
final = pd.concat([rounds, overall_categories], axis=1)
final

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,1,0,0,0,0,0,1,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,0,0,0,0,0,1,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,0,0,0,0,0,1,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88898,91661,/organization/zyrra,Zyrra,USA,MA,Boston,Cambridge,/funding-round/7a0fa791c96c9c447ee54662395beab9,venture,NaN,...,0,0,0,0,0,1,0,0,0,0
88899,91662,/organization/zyrra,Zyrra,USA,MA,Boston,Cambridge,/funding-round/cc8cd96a500332d5b4f4bcbddb306099,angel,NaN,...,0,0,0,0,0,1,0,0,0,0
88900,91663,/organization/zystor,Zystor,USA,WI,Milwaukee,Milwaukee,/funding-round/7ea37c0ff4132b084832039a9380d34e,venture,A,...,0,0,0,0,1,0,0,0,0,0
88901,91664,/organization/zytoprotec,Zytoprotec,AUT,3,Vienna,Gerasdorf Bei Wien,/funding-round/0c43e717298296d2fb334fb236300f6e,venture,A,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
sector1_temp = final[final['Communication Services']==1]
Communication_Services = sector1_temp.groupby(['Communication Services', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Communication_Services['sector_index'] = pd.qcut(Communication_Services['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Communication_Services)

                                              raised_amount_usd sector_index
Communication Services funding_round                                        
1                      angel                       7.163040e+05            3
                       convertible_note            1.110061e+06            3
                       debt_financing              5.381371e+07            8
                       equity_crowdfunding         2.938005e+05            1
                       grant                       5.153579e+05            2
                       non_equity_assistance       1.385747e+05            1
                       post_ipo_debt               1.179186e+08           10
                       post_ipo_equity             5.721367e+07            8
                       private_equity              7.250843e+07            9
                       product_crowdfunding        4.469725e+05            1
                       secondary_market            2.043000e+08           10

In [ ]:
Communication_Services['sector_index'].to_json('sector_rounds_communication_services_v2.json')

In [ ]:
with open('sector_rounds_communication_services_v2.json') as json_file:
    sector_rounds_communication_services_dict = json.load(json_file)

In [ ]:
sector_rounds_communication_services_index = pd.DataFrame.from_dict(sector_rounds_communication_services_dict, orient='index')
sector_rounds_communication_services_index.index = [parse_tuple(item) for item in list(sector_rounds_communication_services_index.index)]

sector_rounds_communication_services_index.index = pd.MultiIndex.from_tuples(sector_rounds_communication_services_index.index)
sector_rounds_communication_services_index.index.names = ['communication_services', 'round']
print(sector_rounds_communication_services_index)


                                               0
communication_services round                    
1                      angel                   3
                       convertible_note        3
                       debt_financing          8
                       equity_crowdfunding     1
                       grant                   2
                       non_equity_assistance   1
                       post_ipo_debt          10
                       post_ipo_equity         8
                       private_equity          9
                       product_crowdfunding    1
                       secondary_market       10
                       seed                    2
                       undisclosed             4
                       venture                 5
                       venture A               4
                       venture B               5
                       venture C               6
                       venture D               6
                    

In [ ]:
sector_rounds_communication_services_index_dict = sector_rounds_communication_services_index.to_dict()[0]
sector_rounds_communication_services_index_dict

{(1, 'angel'): '3',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '8',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '2',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '10',
 (1, 'post_ipo_equity'): '8',
 (1, 'private_equity'): '9',
 (1, 'product_crowdfunding'): '1',
 (1, 'secondary_market'): '10',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '4',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '6',
 (1, 'venture D'): '6',
 (1, 'venture E'): '7',
 (1, 'venture F'): '9',
 (1, 'venture G'): '10',
 (1, 'venture H'): '7'}

In [ ]:
def sec_rnd_communication_ix(row):
    sector = row['Communication Services']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_communication_services_index_dict[search_key]
        return int(value)

final['Communication_Services_round_index'] = final.apply(sec_rnd_communication_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,0,0,0,0,5
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,0,0,0,0,0,1,0,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,0,0,0,0,1,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,0,0,0,0,1,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,0,0,0,1,0,0,0,0,0


### Consumer Discretionary

In [ ]:
sector2_temp = final[final['Consumer Discretionary']==1]
Consumer_Discretionary = sector2_temp.groupby(['Consumer Discretionary', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Consumer_Discretionary['sector_index'] = pd.qcut(Consumer_Discretionary['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Consumer_Discretionary)

                                              raised_amount_usd sector_index
Consumer Discretionary funding_round                                        
1                      angel                       7.525130e+05            2
                       convertible_note            6.391004e+05            2
                       debt_financing              1.382524e+07            5
                       equity_crowdfunding         3.460322e+05            1
                       grant                       4.983615e+06            3
                       non_equity_assistance       2.366403e+05            1
                       post_ipo_debt               2.166667e+07            6
                       post_ipo_equity             7.459290e+07            8
                       private_equity              9.609257e+07            9
                       product_crowdfunding        1.313310e+06            3
                       secondary_market            1.897874e+08           10

In [ ]:
Consumer_Discretionary['sector_index'].to_json('sector_rounds_consumer_discretionary_v2.json')

In [ ]:
with open('sector_rounds_consumer_discretionary_v2.json') as json_file:
    sector_rounds_consumer_discretionary_dict = json.load(json_file)

In [ ]:
sector_rounds_consumer_discretionary_index = pd.DataFrame.from_dict(sector_rounds_consumer_discretionary_dict, orient='index')
sector_rounds_consumer_discretionary_index.index = [parse_tuple(item) for item in list(sector_rounds_consumer_discretionary_index.index)]

sector_rounds_consumer_discretionary_index.index = pd.MultiIndex.from_tuples(sector_rounds_consumer_discretionary_index.index)
sector_rounds_consumer_discretionary_index.index.names = ['consumer_discretionary', 'round']
print(sector_rounds_consumer_discretionary_index)


                                               0
consumer_discretionary round                    
1                      angel                   2
                       convertible_note        2
                       debt_financing          5
                       equity_crowdfunding     1
                       grant                   3
                       non_equity_assistance   1
                       post_ipo_debt           6
                       post_ipo_equity         8
                       private_equity          9
                       product_crowdfunding    3
                       secondary_market       10
                       seed                    1
                       undisclosed             4
                       venture                 5
                       venture A               4
                       venture B               6
                       venture C               7
                       venture D               7
                    

In [ ]:
sector_rounds_consumer_discretionary_index_dict = sector_rounds_consumer_discretionary_index.to_dict()[0]
sector_rounds_consumer_discretionary_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '2',
 (1, 'debt_financing'): '5',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '6',
 (1, 'post_ipo_equity'): '8',
 (1, 'private_equity'): '9',
 (1, 'product_crowdfunding'): '3',
 (1, 'secondary_market'): '10',
 (1, 'seed'): '1',
 (1, 'undisclosed'): '4',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '6',
 (1, 'venture C'): '7',
 (1, 'venture D'): '7',
 (1, 'venture E'): '9',
 (1, 'venture F'): '8',
 (1, 'venture G'): '10',
 (1, 'venture H'): '10'}

In [ ]:
def sec_rnd_consumer_discretionary_ix(row):
    sector = row['Consumer Discretionary']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_consumer_discretionary_index_dict[search_key]
        return int(value)

final['Consumer_Discretionary_round_index'] = final.apply(sec_rnd_consumer_discretionary_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,0,0,0,5,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,0,0,0,0,1,0,0,0,0,6
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,0,0,0,1,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,0,0,0,1,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,0,0,1,0,0,0,0,0,0


### Consumer Stapels

In [ ]:
sector3_temp = final[final['Consumer Staples']==1]
Consumer_Staples = sector3_temp.groupby(['Consumer Staples', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Consumer_Staples['raised_amount_usd'] = Consumer_Staples['raised_amount_usd'].fillna(0)
Consumer_Staples['sector_index'] = pd.qcut(Consumer_Staples['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
#Consumer_Staples = Consumer_Staples['raised_amount_usd'].fillna(0)
print(Consumer_Staples)

                                        raised_amount_usd sector_index
Consumer Staples funding_round                                        
1                angel                       8.254352e+05            3
                 convertible_note            1.043660e+06            3
                 debt_financing              5.095621e+07            8
                 equity_crowdfunding         3.998366e+05            2
                 grant                       9.737643e+04            1
                 non_equity_assistance       8.000000e+04            1
                 post_ipo_debt               5.450000e+07            9
                 post_ipo_equity             6.620836e+07           10
                 private_equity              4.562768e+07            8
                 product_crowdfunding        1.647166e+06            4
                 secondary_market            0.000000e+00            1
                 seed                        6.837995e+05            2
      

In [ ]:
Consumer_Staples['sector_index'].to_json('sector_rounds_consumer_staples_v2.json')

In [ ]:
with open('sector_rounds_consumer_staples_v2.json') as json_file:
    sector_rounds_consumer_staples_dict = json.load(json_file)

In [ ]:
sector_rounds_consumer_staples_index = pd.DataFrame.from_dict(sector_rounds_consumer_staples_dict, orient='index')
sector_rounds_consumer_staples_index.index = [parse_tuple(item) for item in list(sector_rounds_consumer_staples_index.index)]

sector_rounds_consumer_staples_index.index = pd.MultiIndex.from_tuples(sector_rounds_consumer_staples_index.index)
sector_rounds_consumer_staples_index.index.names = ['consumer_staples', 'round']
print(sector_rounds_consumer_staples_index)


                                         0
consumer_staples round                    
1                angel                   3
                 convertible_note        3
                 debt_financing          8
                 equity_crowdfunding     2
                 grant                   1
                 non_equity_assistance   1
                 post_ipo_debt           9
                 post_ipo_equity        10
                 private_equity          8
                 product_crowdfunding    4
                 secondary_market        1
                 seed                    2
                 undisclosed             5
                 venture                 5
                 venture A               4
                 venture B               6
                 venture C               6
                 venture D              10
                 venture E               9
                 venture F               7
                 venture G               7


In [ ]:
sector_rounds_consumer_staples_index_dict = sector_rounds_consumer_staples_index.to_dict()[0]
sector_rounds_consumer_staples_index_dict

{(1, 'angel'): '3',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '8',
 (1, 'equity_crowdfunding'): '2',
 (1, 'grant'): '1',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '9',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '8',
 (1, 'product_crowdfunding'): '4',
 (1, 'secondary_market'): '1',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '5',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '6',
 (1, 'venture C'): '6',
 (1, 'venture D'): '10',
 (1, 'venture E'): '9',
 (1, 'venture F'): '7',
 (1, 'venture G'): '7'}

In [ ]:
def sec_rnd_consumer_staples_ix(row):
    sector = row['Consumer Staples']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_consumer_staples_index_dict[search_key]
        return int(value)

final['Consumer_Staples_round_index'] = final.apply(sec_rnd_consumer_staples_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,0,0,5,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,0,0,0,1,0,0,0,0,6,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,0,0,1,0,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,0,0,1,0,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,0,1,0,0,0,0,0,0,0


### Energy

In [ ]:
sector4_temp = final[final['Energy']==1]
Energy = sector4_temp.groupby(['Energy', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Energy['raised_amount_usd'] = Energy['raised_amount_usd'].fillna(0)
Energy['sector_index'] = pd.qcut(Energy['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
#Consumer_Staples = Consumer_Staples['raised_amount_usd'].fillna(0)
print(Energy)

                              raised_amount_usd sector_index
Energy funding_round                                        
1      angel                       6.598609e+05            1
       convertible_note            1.719649e+06            3
       debt_financing              2.704158e+07            7
       equity_crowdfunding         9.491204e+05            2
       grant                       1.206157e+07            5
       non_equity_assistance       2.478055e+06            3
       post_ipo_debt               1.761250e+07            6
       post_ipo_equity             7.686734e+07           10
       private_equity              1.018016e+08           10
       product_crowdfunding        4.630059e+05            1
       secondary_market            0.000000e+00            1
       seed                        6.944901e+05            2
       undisclosed                 3.143131e+07            8
       venture                     1.186847e+07            4
       venture A        

In [ ]:
Energy['sector_index'].to_json('sector_rounds_energy_v2.json')

In [ ]:
with open('sector_rounds_energy_v2.json') as json_file:
    sector_rounds_energy_dict = json.load(json_file)

In [ ]:
sector_rounds_energy_index = pd.DataFrame.from_dict(sector_rounds_energy_dict, orient='index')
sector_rounds_energy_index.index = [parse_tuple(item) for item in list(sector_rounds_energy_index.index)]

sector_rounds_energy_index.index = pd.MultiIndex.from_tuples(sector_rounds_energy_index.index)
sector_rounds_energy_index.index.names = ['energy', 'round']
print(sector_rounds_energy_index)


                               0
energy round                    
1      angel                   1
       convertible_note        3
       debt_financing          7
       equity_crowdfunding     2
       grant                   5
       non_equity_assistance   3
       post_ipo_debt           6
       post_ipo_equity        10
       private_equity         10
       product_crowdfunding    1
       secondary_market        1
       seed                    2
       undisclosed             8
       venture                 4
       venture A               4
       venture B               6
       venture C               7
       venture D               9
       venture E               9
       venture F               8
       venture G               5


In [ ]:
sector_rounds_energy_index_dict = sector_rounds_energy_index.to_dict()[0]
sector_rounds_energy_index_dict

{(1, 'angel'): '1',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '7',
 (1, 'equity_crowdfunding'): '2',
 (1, 'grant'): '5',
 (1, 'non_equity_assistance'): '3',
 (1, 'post_ipo_debt'): '6',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '10',
 (1, 'product_crowdfunding'): '1',
 (1, 'secondary_market'): '1',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '8',
 (1, 'venture'): '4',
 (1, 'venture A'): '4',
 (1, 'venture B'): '6',
 (1, 'venture C'): '7',
 (1, 'venture D'): '9',
 (1, 'venture E'): '9',
 (1, 'venture F'): '8',
 (1, 'venture G'): '5'}

In [ ]:
def sec_rnd_energy_ix(row):
    sector = row['Energy']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_energy_index_dict[search_key]
        return int(value)

final['Energy_round_index'] = final.apply(sec_rnd_energy_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,0,5,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,0,0,1,0,0,0,0,6,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,0,1,0,0,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,0,1,0,0,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,1,0,0,0,0,0,0,0,0


### Financials

In [ ]:
sector5_temp = final[final['Financials']==1]
Financials = sector5_temp.groupby(['Financials', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Financials['sector_index'] = pd.qcut(Financials['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Financials)

                                  raised_amount_usd sector_index
Financials funding_round                                        
1          angel                       8.793897e+05            2
           convertible_note            6.262366e+05            1
           debt_financing              2.230856e+07            5
           equity_crowdfunding         6.027401e+05            1
           grant                       9.516473e+05            3
           non_equity_assistance       6.517808e+04            1
           post_ipo_debt               1.068762e+09           10
           post_ipo_equity             3.325639e+07            7
           private_equity              8.344521e+07            9
           product_crowdfunding        9.731476e+05            3
           secondary_market            3.273333e+07            6
           seed                        6.984748e+05            2
           undisclosed                 3.912588e+07            8
           venture       

In [ ]:
Financials['sector_index'].to_json('sector_rounds_financials_v2.json')

In [ ]:
with open('sector_rounds_financials_v2.json') as json_file:
    sector_rounds_financials_dict = json.load(json_file)

In [ ]:
sector_rounds_financials_index = pd.DataFrame.from_dict(sector_rounds_financials_dict, orient='index')
sector_rounds_financials_index.index = [parse_tuple(item) for item in list(sector_rounds_financials_index.index)]

sector_rounds_financials_index.index = pd.MultiIndex.from_tuples(sector_rounds_financials_index.index)
sector_rounds_financials_index.index.names = ['financials', 'round']
print(sector_rounds_financials_index)


                               0
energy round                    
1      angel                   2
       convertible_note        1
       debt_financing          5
       equity_crowdfunding     1
       grant                   3
       non_equity_assistance   1
       post_ipo_debt          10
       post_ipo_equity         7
       private_equity          9
       product_crowdfunding    3
       secondary_market        6
       seed                    2
       undisclosed             8
       venture                 4
       venture A               4
       venture B               5
       venture C               6
       venture D               8
       venture E              10
       venture F              10
       venture G               7
       venture H               9


In [ ]:
sector_rounds_financials_index_dict = sector_rounds_financials_index.to_dict()[0]
sector_rounds_financials_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '1',
 (1, 'debt_financing'): '5',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '10',
 (1, 'post_ipo_equity'): '7',
 (1, 'private_equity'): '9',
 (1, 'product_crowdfunding'): '3',
 (1, 'secondary_market'): '6',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '8',
 (1, 'venture'): '4',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '6',
 (1, 'venture D'): '8',
 (1, 'venture E'): '10',
 (1, 'venture F'): '10',
 (1, 'venture G'): '7',
 (1, 'venture H'): '9'}

In [ ]:
def sec_rnd_financials_ix(row):
    sector = row['Financials']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_financials_index_dict[search_key]
        return int(value)

final['Financials_round_index'] = final.apply(sec_rnd_financials_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,0,5,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,0,1,0,0,0,0,6,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,0,1,0,0,0,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,0,1,0,0,0,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,1,0,0,0,0,0,0,0,0,0


### Health Care

In [ ]:
sector6_temp = final[final['Health Care']==1]
Health_Care = sector6_temp.groupby(['Health Care', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Health_Care['sector_index'] = pd.qcut(Health_Care['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Health_Care)

                                   raised_amount_usd sector_index
Health Care funding_round                                        
1           angel                       9.646572e+05            2
            convertible_note            2.027768e+06            3
            debt_financing              1.170610e+07            5
            equity_crowdfunding         4.307165e+05            1
            grant                       4.852153e+06            3
            non_equity_assistance       2.174738e+05            1
            post_ipo_debt               5.486272e+07           10
            post_ipo_equity             4.096715e+07           10
            private_equity              4.460004e+07           10
            product_crowdfunding        9.160452e+05            2
            secondary_market            3.976667e+07            9
            seed                        8.442865e+05            1
            undisclosed                 5.254965e+06            4
          

In [ ]:
Health_Care['sector_index'].to_json('sector_rounds_health_care_v2.json')

In [ ]:
with open('sector_rounds_health_care_v2.json') as json_file:
    sector_rounds_health_care_dict = json.load(json_file)

In [ ]:
sector_rounds_health_care_index = pd.DataFrame.from_dict(sector_rounds_health_care_dict, orient='index')
sector_rounds_health_care_index.index = [parse_tuple(item) for item in list(sector_rounds_health_care_index.index)]

sector_rounds_health_care_index.index = pd.MultiIndex.from_tuples(sector_rounds_health_care_index.index)
sector_rounds_health_care_index.index.names = ['Health Care', 'round']
print(sector_rounds_health_care_index)


                               0
energy round                    
1      angel                   2
       convertible_note        3
       debt_financing          5
       equity_crowdfunding     1
       grant                   3
       non_equity_assistance   1
       post_ipo_debt          10
       post_ipo_equity        10
       private_equity         10
       product_crowdfunding    2
       secondary_market        9
       seed                    1
       undisclosed             4
       venture                 4
       venture A               5
       venture B               6
       venture C               6
       venture D               7
       venture E               7
       venture F               8
       venture G               8
       venture H               9


In [ ]:
sector_rounds_health_care_index_dict = sector_rounds_health_care_index.to_dict()[0]
sector_rounds_health_care_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '5',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '10',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '10',
 (1, 'product_crowdfunding'): '2',
 (1, 'secondary_market'): '9',
 (1, 'seed'): '1',
 (1, 'undisclosed'): '4',
 (1, 'venture'): '4',
 (1, 'venture A'): '5',
 (1, 'venture B'): '6',
 (1, 'venture C'): '6',
 (1, 'venture D'): '7',
 (1, 'venture E'): '7',
 (1, 'venture F'): '8',
 (1, 'venture G'): '8',
 (1, 'venture H'): '9'}

In [ ]:
def sec_rnd_health_care_ix(row):
    sector = row['Health Care']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_health_care_index_dict[search_key]
        return int(value)

final['Health_Care_round_index'] = final.apply(sec_rnd_health_care_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,...,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,...,0,0,0,0,5,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,...,1,0,0,0,0,6,0,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,...,1,0,0,0,0,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,...,1,0,0,0,0,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,...,0,0,0,0,0,0,0,0,0,0


### Industrials

In [ ]:
sector7_temp = final[final['Industrials']==1]
Industrials = sector7_temp.groupby(['Industrials', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Industrials['sector_index'] = pd.qcut(Industrials['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Industrials)

                                   raised_amount_usd sector_index
Industrials funding_round                                        
1           angel                       8.428266e+05            2
            convertible_note            6.765369e+05            1
            debt_financing              1.645734e+07            5
            equity_crowdfunding         4.565184e+05            1
            grant                       2.181093e+06            3
            non_equity_assistance       3.281696e+05            1
            post_ipo_debt               6.964428e+07            7
            post_ipo_equity             2.123767e+08           10
            private_equity              9.639480e+07            9
            product_crowdfunding        1.177786e+06            3
            secondary_market            1.251756e+08            9
            seed                        6.793114e+05            2
            undisclosed                 3.912437e+07            6
          

In [ ]:
Industrials['sector_index'].to_json('sector_rounds_industrials_v2.json')

In [ ]:
with open('sector_rounds_industrials_v2.json') as json_file:
    sector_rounds_industrials_dict = json.load(json_file)

In [ ]:
sector_rounds_industrials_index = pd.DataFrame.from_dict(sector_rounds_industrials_dict, orient='index')
sector_rounds_industrials_index.index = [parse_tuple(item) for item in list(sector_rounds_industrials_index.index)]

sector_rounds_industrials_index.index = pd.MultiIndex.from_tuples(sector_rounds_industrials_index.index)
sector_rounds_industrials_index.index.names = ['Industrials', 'round']
print(sector_rounds_industrials_index)


                                    0
Industrials round                    
1           angel                   2
            convertible_note        1
            debt_financing          5
            equity_crowdfunding     1
            grant                   3
            non_equity_assistance   1
            post_ipo_debt           7
            post_ipo_equity        10
            private_equity          9
            product_crowdfunding    3
            secondary_market        9
            seed                    2
            undisclosed             6
            venture                 4
            venture A               4
            venture B               5
            venture C               6
            venture D               7
            venture E               8
            venture F               8
            venture G              10
            venture H              10


In [ ]:
sector_rounds_industrials_index_dict = sector_rounds_industrials_index.to_dict()[0]
sector_rounds_industrials_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '1',
 (1, 'debt_financing'): '5',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '7',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '9',
 (1, 'product_crowdfunding'): '3',
 (1, 'secondary_market'): '9',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '6',
 (1, 'venture'): '4',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '6',
 (1, 'venture D'): '7',
 (1, 'venture E'): '8',
 (1, 'venture F'): '8',
 (1, 'venture G'): '10',
 (1, 'venture H'): '10'}

In [ ]:
def sec_rnd_industrials_ix(row):
    sector = row['Industrials']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_industrials_index_dict[search_key]
        return int(value)

final['Industrials_round_index'] = final.apply(sec_rnd_industrials_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,company_category_list,company_sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index,Industrials_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B,[Media],[Communication Services],1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B,"[Apps, Games, Mobile]","[Information Technology, Consumer Discretionary]",0,1,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,[Curated Web],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture,[Software],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed,[E-Commerce],[Industrials],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2


### Information Technology

In [ ]:
sector8_temp = final[final['Information Technology']==1]
Information_Technology = sector8_temp.groupby(['Information Technology', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Information_Technology['sector_index'] = pd.qcut(Information_Technology['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Information_Technology)

                                              raised_amount_usd sector_index
Information Technology funding_round                                        
1                      angel                       7.566352e+05            2
                       convertible_note            1.291250e+06            3
                       debt_financing              2.285428e+07            6
                       equity_crowdfunding         6.176385e+05            1
                       grant                       1.409909e+06            3
                       non_equity_assistance       2.324847e+05            1
                       post_ipo_debt               1.290447e+08           10
                       post_ipo_equity             1.703139e+08           10
                       private_equity              5.546558e+07            8
                       product_crowdfunding        1.164837e+06            2
                       secondary_market            9.154808e+07           10

In [ ]:
Information_Technology['sector_index'].to_json('sector_rounds_information_technology_v2.json')

In [ ]:
with open('sector_rounds_information_technology_v2.json') as json_file:
    sector_rounds_information_technology_dict = json.load(json_file)

In [ ]:
sector_rounds_information_technology_index = pd.DataFrame.from_dict(sector_rounds_information_technology_dict, orient='index')
sector_rounds_information_technology_index.index = [parse_tuple(item) for item in list(sector_rounds_industrials_index.index)]

sector_rounds_information_technology_index.index = pd.MultiIndex.from_tuples(sector_rounds_information_technology_index.index)
sector_rounds_information_technology_index.index.names = ['Information Technology', 'round']
print(sector_rounds_information_technology_index)


                                    0
Industrials round                    
1           angel                   2
            convertible_note        3
            debt_financing          6
            equity_crowdfunding     1
            grant                   3
            non_equity_assistance   1
            post_ipo_debt          10
            post_ipo_equity        10
            private_equity          8
            product_crowdfunding    2
            secondary_market       10
            seed                    1
            undisclosed             4
            venture                 5
            venture A               4
            venture B               5
            venture C               6
            venture D               7
            venture E               7
            venture F               8
            venture G               9
            venture H               9


In [ ]:
sector_rounds_information_technology_index_dict = sector_rounds_information_technology_index.to_dict()[0]
sector_rounds_information_technology_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '6',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '10',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '8',
 (1, 'product_crowdfunding'): '2',
 (1, 'secondary_market'): '10',
 (1, 'seed'): '1',
 (1, 'undisclosed'): '4',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '6',
 (1, 'venture D'): '7',
 (1, 'venture E'): '7',
 (1, 'venture F'): '8',
 (1, 'venture G'): '9',
 (1, 'venture H'): '9'}

In [ ]:
def sec_rnd_information_technology_ix(row):
    sector = row['Information Technology']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_information_technology_index_dict[search_key]
        return int(value)

final['Information_Technology_round_index'] = final.apply(sec_rnd_information_technology_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,company_category_list,company_sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index,Industrials_round_index,Information_Technology_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B,[Media],[Communication Services],1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B,"[Apps, Games, Mobile]","[Information Technology, Consumer Discretionary]",0,1,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0,5
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,[Curated Web],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture,[Software],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed,[E-Commerce],[Industrials],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0


### Materials

In [ ]:
sector9_temp = final[final['Materials']==1]
Materials = sector9_temp.groupby(['Materials', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Materials['sector_index'] = pd.qcut(Materials['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Materials)

                                raised_amount_usd sector_index
Materials funding_round                                       
1         angel                      5.330151e+05            1
          convertible_note           3.507415e+05            1
          debt_financing             6.484216e+07           10
          equity_crowdfunding        6.825129e+05            2
          grant                      3.508810e+06            3
          post_ipo_debt              2.500000e+07            7
          post_ipo_equity            1.399839e+07            5
          private_equity             4.788339e+07           10
          product_crowdfunding       1.139333e+06            3
          seed                       6.285549e+05            2
          undisclosed                1.686422e+07            7
          venture                    6.386938e+06            4
          venture A                  5.139723e+06            4
          venture B                  1.284904e+07      

In [ ]:
Materials['sector_index'].to_json('sector_rounds_materials_v2.json')

In [ ]:
with open('sector_rounds_materials_v2.json') as json_file:
    sector_rounds_materials_dict = json.load(json_file)

In [ ]:
sector_rounds_materials_index = pd.DataFrame.from_dict(sector_rounds_materials_dict, orient='index')
sector_rounds_materials_index.index = [parse_tuple(item) for item in list(sector_rounds_materials_index.index)]

sector_rounds_materials_index.index = pd.MultiIndex.from_tuples(sector_rounds_materials_index.index)
sector_rounds_materials_index.index.names = ['Materials', 'round']
print(sector_rounds_materials_index)


                                 0
Materials round                   
1         angel                  1
          convertible_note       1
          debt_financing        10
          equity_crowdfunding    2
          grant                  3
          post_ipo_debt          7
          post_ipo_equity        5
          private_equity        10
          product_crowdfunding   3
          seed                   2
          undisclosed            7
          venture                4
          venture A              4
          venture B              5
          venture C              8
          venture D              9
          venture E              6
          venture F              9
          venture G              8


In [ ]:
sector_rounds_materials_index_dict = sector_rounds_materials_index.to_dict()[0]
sector_rounds_materials_index_dict

{(1, 'angel'): '1',
 (1, 'convertible_note'): '1',
 (1, 'debt_financing'): '10',
 (1, 'equity_crowdfunding'): '2',
 (1, 'grant'): '3',
 (1, 'post_ipo_debt'): '7',
 (1, 'post_ipo_equity'): '5',
 (1, 'private_equity'): '10',
 (1, 'product_crowdfunding'): '3',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '7',
 (1, 'venture'): '4',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '8',
 (1, 'venture D'): '9',
 (1, 'venture E'): '6',
 (1, 'venture F'): '9',
 (1, 'venture G'): '8'}

In [ ]:
def sec_rnd_materials_ix(row):
    sector = row['Materials']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_materials_index_dict[search_key]
        return int(value)

final['Materials_round_index'] = final.apply(sec_rnd_materials_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,company_category_list,company_sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index,Industrials_round_index,Information_Technology_round_index,Materials_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B,[Media],[Communication Services],1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B,"[Apps, Games, Mobile]","[Information Technology, Consumer Discretionary]",0,1,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0,5,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,[Curated Web],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture,[Software],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed,[E-Commerce],[Industrials],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0


### Real Estate

In [ ]:
sector10_temp = final[final['Real Estate']==1]
Real_Estate = sector10_temp.groupby(['Real Estate', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Real_Estate['sector_index'] = pd.qcut(Real_Estate['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Real_Estate)

                                   raised_amount_usd sector_index
Real Estate funding_round                                        
1           angel                       6.990287e+05            2
            convertible_note            5.453623e+05            1
            debt_financing              2.007807e+07            6
            equity_crowdfunding         8.174489e+05            2
            grant                       1.214109e+06            3
            non_equity_assistance       3.332057e+04            1
            post_ipo_debt               1.561000e+08           10
            post_ipo_equity             1.897631e+08           10
            private_equity              8.432144e+07            9
            product_crowdfunding        3.708026e+06            3
            secondary_market            6.000000e+06            4
            seed                        6.080774e+05            1
            undisclosed                 2.277504e+07            6
          

In [ ]:
Real_Estate['sector_index'].to_json('sector_rounds_real_estate_v2.json')

In [ ]:
with open('sector_rounds_real_estate_v2.json') as json_file:
    sector_rounds_real_estate_dict = json.load(json_file)

In [ ]:
sector_rounds_real_estate_index = pd.DataFrame.from_dict(sector_rounds_real_estate_dict, orient='index')
sector_rounds_real_estate_index.index = [parse_tuple(item) for item in list(sector_rounds_real_estate_index.index)]

sector_rounds_real_estate_index.index = pd.MultiIndex.from_tuples(sector_rounds_real_estate_index.index)
sector_rounds_real_estate_index.index.names = ['Real Estate', 'round']
print(sector_rounds_real_estate_index)


                                    0
Real Estate round                    
1           angel                   2
            convertible_note        1
            debt_financing          6
            equity_crowdfunding     2
            grant                   3
            non_equity_assistance   1
            post_ipo_debt          10
            post_ipo_equity        10
            private_equity          9
            product_crowdfunding    3
            secondary_market        4
            seed                    1
            undisclosed             6
            venture                 5
            venture A               4
            venture B               5
            venture C               7
            venture D               8
            venture E               8
            venture F               7
            venture G               9


In [ ]:
sector_rounds_real_estate_index_dict = sector_rounds_real_estate_index.to_dict()[0]
sector_rounds_real_estate_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '1',
 (1, 'debt_financing'): '6',
 (1, 'equity_crowdfunding'): '2',
 (1, 'grant'): '3',
 (1, 'non_equity_assistance'): '1',
 (1, 'post_ipo_debt'): '10',
 (1, 'post_ipo_equity'): '10',
 (1, 'private_equity'): '9',
 (1, 'product_crowdfunding'): '3',
 (1, 'secondary_market'): '4',
 (1, 'seed'): '1',
 (1, 'undisclosed'): '6',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '5',
 (1, 'venture C'): '7',
 (1, 'venture D'): '8',
 (1, 'venture E'): '8',
 (1, 'venture F'): '7',
 (1, 'venture G'): '9'}

In [ ]:
def sec_rnd_real_estate_ix(row):
    sector = row['Real Estate']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_real_estate_index_dict[search_key]
        return int(value)

final['Real_Estate_round_index'] = final.apply(sec_rnd_real_estate_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,company_category_list,company_sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index,Industrials_round_index,Information_Technology_round_index,Materials_round_index,Real_Estate_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B,[Media],[Communication Services],1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B,"[Apps, Games, Mobile]","[Information Technology, Consumer Discretionary]",0,1,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0,5,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,[Curated Web],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture,[Software],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed,[E-Commerce],[Industrials],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0


### Utilities

In [ ]:
sector11_temp = final[final['Utilities']==1]
Utilities = sector11_temp.groupby(['Utilities', 'funding_round']).agg({'raised_amount_usd': 'mean'})
Utilities['sector_index'] = pd.qcut(Utilities['raised_amount_usd'], q = 10, labels=['1','2','3','4','5','6','7','8','9','10'])
print(Utilities)

                                 raised_amount_usd sector_index
Utilities funding_round                                        
1         angel                       6.152697e+05            2
          convertible_note            1.916626e+06            3
          debt_financing              6.191909e+07           10
          equity_crowdfunding         4.635461e+05            1
          grant                       1.750185e+07            6
          non_equity_assistance       8.829000e+05            3
          post_ipo_equity             4.261962e+07            9
          private_equity              8.216547e+07           10
          product_crowdfunding        1.500300e+04            1
          seed                        5.510443e+05            2
          undisclosed                 2.339938e+07            6
          venture                     1.227787e+07            5
          venture A                   6.574958e+06            4
          venture B                   2.

In [ ]:
Utilities['sector_index'].to_json('sector_rounds_utilities_v2.json')

In [ ]:
with open('sector_rounds_utilities_v2.json') as json_file:
    sector_rounds_utilities_dict = json.load(json_file)

In [ ]:
sector_rounds_utilities_index = pd.DataFrame.from_dict(sector_rounds_utilities_dict, orient='index')
sector_rounds_utilities_index.index = [parse_tuple(item) for item in list(sector_rounds_utilities_index.index)]

sector_rounds_utilities_index.index = pd.MultiIndex.from_tuples(sector_rounds_utilities_index.index)
sector_rounds_utilities_index.index.names = ['Utilities', 'round']
print(sector_rounds_utilities_index)


                                  0
Utilities round                    
1         angel                   2
          convertible_note        3
          debt_financing         10
          equity_crowdfunding     1
          grant                   6
          non_equity_assistance   3
          post_ipo_equity         9
          private_equity         10
          product_crowdfunding    1
          seed                    2
          undisclosed             6
          venture                 5
          venture A               4
          venture B               8
          venture C               7
          venture D               9
          venture E               8
          venture F               5


In [ ]:
sector_rounds_utilities_index_dict = sector_rounds_utilities_index.to_dict()[0]
sector_rounds_utilities_index_dict

{(1, 'angel'): '2',
 (1, 'convertible_note'): '3',
 (1, 'debt_financing'): '10',
 (1, 'equity_crowdfunding'): '1',
 (1, 'grant'): '6',
 (1, 'non_equity_assistance'): '3',
 (1, 'post_ipo_equity'): '9',
 (1, 'private_equity'): '10',
 (1, 'product_crowdfunding'): '1',
 (1, 'seed'): '2',
 (1, 'undisclosed'): '6',
 (1, 'venture'): '5',
 (1, 'venture A'): '4',
 (1, 'venture B'): '8',
 (1, 'venture C'): '7',
 (1, 'venture D'): '9',
 (1, 'venture E'): '8',
 (1, 'venture F'): '5'}

In [ ]:
def sec_rnd_utilities_ix(row):
    sector = row['Utilities']
    comm_round = row['funding_round']

    if sector == 0:
        return int(0)
    else:
        search_key = tuple((sector, comm_round))
        value = sector_rounds_real_estate_index_dict[search_key]
        return int(value)

final['Utilities_round_index'] = final.apply(sec_rnd_utilities_ix, axis=1)
final.head()

,Unnamed: 0,company_permalink,company_name,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,funding_round,company_category_list,company_sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities,Communication_Services_round_index,Consumer_Discretionary_round_index,Consumer_Staples_round_index,Energy_round_index,Financials_round_index,Health_Care_round_index,Industrials_round_index,Information_Technology_round_index,Materials_round_index,Real_Estate_round_index,Utilities_round_index
0,0,/organization/-fame,#fame,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,2015-01-05,10000000.0,venture B,[Media],[Communication Services],1,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
1,1,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,2014-01-30,3406878.0,venture B,"[Apps, Games, Mobile]","[Information Technology, Consumer Discretionary]",0,1,0,0,0,0,0,1,0,0,0,0,6,0,0,0,0,0,5,0,0,0
2,2,/organization/0-6-com,0-6.com,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,2008-03-19,2000000.0,venture A,[Curated Web],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0
3,3,/organization/004-technologies,004 Technologies,USA,IL,"Springfield, Illinois",Champaign,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,2014-07-24,NaN,venture,[Software],[Information Technology],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,0,0,0
4,4,/organization/1-2-3-listo,"1,2,3 Listo",CHL,12,Santiago,Las Condes,/funding-round/6de4609e894495105bc791ed64361288,seed,NaN,2013-02-18,40000.0,seed,[E-Commerce],[Industrials],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [ ]:
final.columns

Index(['Unnamed: 0', 'company_permalink', 'company_name',
       'company_country_code', 'company_state_code', 'company_region',
       'company_city', 'funding_round_permalink', 'funding_round_type',
       'funding_round_code', 'funded_at', 'raised_amount_usd', 'funding_round',
       'company_category_list', 'company_sector', 'Communication Services',
       'Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials',
       'Health Care', 'Industrials', 'Information Technology', 'Materials',
       'Real Estate', 'Utilities', 'Communication_Services_round_index',
       'Consumer_Discretionary_round_index', 'Consumer_Staples_round_index',
       'Energy_round_index', 'Financials_round_index',
       'Health_Care_round_index', 'Industrials_round_index',
       'Information_Technology_round_index', 'Materials_round_index',
       'Real_Estate_round_index', 'Utilities_round_index'],
      dtype='object')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f906c6b7-4fb3-42a0-9ffa-f1056dea520e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>